In [47]:
import plotly.graph_objects as go
from IPython.core.display import display, HTML
import pandas as pd

C:\Users\Yuxuan Zheng\AppData\Local\Temp\ipykernel_25680\241899341.py:2: DeprecationWarning:

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display



In [48]:
conc = 15

In [49]:
import numpy as np
from scipy.integrate import odeint
from tqdm import tqdm
import time
import random

# Constants
a21 = 1
a31 = 0.27
a32 = 0.73

a41 = 0.18
a42 = 0.24
a43 = 0.58

a51 = 0.24
a52 = 0.23
a53 = 0.2
a54 = 0.33

W2 = 63000
W3 = 20000
W4 = 15000
W5 = 33000
Ws2 = 8000

# P980 = 3*10**4  # our PD

def system(state, t, x, P980, c1, c2, c3, c4, k31, k41, k51):

    ns2, n1, n2, n3, n4, n5 = state

    #ns2
    ms2 = 1.23*P980*(37*(100-x)-ns2) - Ws2*ns2 - (c1*n1+c2*n2+c3*n3+c4*n4)*ns2  # ns1 = total_Yb - ns2

    # n1
    m1 = -c1*n1*ns2 + a21*W2*n2 + a31*W3*n3 + a41*W4*n4 + a51*W5*n5 - k41*n1*n4 - k31*n1*n3 - k51*n5*n1

    # n2
    m2 = c1*n1*ns2 - c2*n2*ns2 - a21*W2*n2 + a32*W3*n3 + a42*W4*n4 + a52*W5*n5 + k41*n1*n4 + 2*k31*n1*n3

    # n3
    m3 = c2*n2*ns2 - c3*n3*ns2 - (a31+a32)*W3*n3 + a43*W4*n4 + a53*W5*n5 + 2*k51*n1*n5 + k41*n1*n4 - k31*n1*n3

    # n4
    m4 = c3*n3*ns2 - c4*n4*ns2 - (a43+a42+a41)*W4*n4 + a54*W5*n5 - k41*n1*n4

    # n5
    m5 = c4*n4*ns2 - (a54+a53+a52+a51)*W5*n5 - k51*n1*n5


    return [ms2, m1, m2, m3, m4, m5]

In [50]:
from scipy.integrate import odeint
from tqdm import tqdm

def compute_values_for_x(x, P980, c1, c2, c3, c4, k31, k41, k51):

    # Compute initial condition
    y = 37 * x
    state0 = [0, y, 0, 0, 0, 0]

    # ODEs
    t = np.arange(0.0, 0.001, 0.000001)

    state = odeint(system, state0, t, args=(x, P980, c1, c2, c3, c4, k31, k41, k51))

    # Compute NIR and blue_total
    NIR = a31 * W3 * state[:, 3][-1]
    blue_1 = a41 * W4 * state[:, 4][-1]
    blue_2 = a52 * W5 * state[:, 5][-1]
    blue_total = blue_1 + blue_2

    return NIR, blue_total


# x_values = [4, 6, 8, 10, 12, 15, 50]  # Range of C2 values from 2% to 40%


In [51]:
p10_blue = [2.8*10**3, 4.4*10**3, 5*10**3, 6*10**3, 7.5*10**3, 9.1*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]
p10_nir = [1.2*10**3, 1.8*10**3, 2.8*10**3, 4.4*10**3, 5*10**3, 6*10**3, 7.5*10**3, 9.1*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]
p12_nir = [1.2*10**3, 1.8*10**3, 2.8*10**3, 4.4*10**3, 5*10**3, 6*10**3, 7.5*10**3, 9.1*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]
p12_blue = [4.4*10**3, 5*10**3, 6*10**3, 7.5*10**3, 9.1*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]
p15_nir = [1.2*10**3, 1.8*10**3, 2.8*10**3, 4.4*10**3, 5*10**3, 6*10**3, 7.5*10**3, 9.1*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]
p15_blue = [4.4*10**3, 5*10**3, 6*10**3, 7.5*10**3, 9.1*10**3, 1.1*10**4, 1.4*10**4, 1.7*10**4, 2*10**4, 2.3*10**4, 2.7*10**4, 3*10**4]

power = {10: (p10_nir, p10_blue), 12: (p12_nir, p12_blue), 15: (p15_nir, p15_blue)}

In [52]:
NIR_values=[]
blue_total_values =[]

count = 0

# gd
# c1_exps, c2_exps, c3_exps, c4_exps, k31_exps, k41_exps, k51_exps = 1.97152332e+00, -1.53795231e+00,  1.24860254e-01,  2.01763255e+00, 5.12551816e-03,  1.98675193e+00, -9.67969325e-04
# gd randomized
c1_exps, c2_exps, c3_exps, c4_exps, k31_exps, k41_exps, k51_exps = 1.95842614, -1.53327474,  0.16847893,  2.01422806,  0.00539746,  1.97773042, -0.00252654
c1, c2, c3, c4, k31, k41, k51 = 10**c1_exps, 10**c2_exps, 10**c3_exps, 10**c4_exps, 10**k31_exps, 10**k41_exps, 10**k51_exps
for index, x in enumerate(power[conc][0]):
    NIR, _ = compute_values_for_x(conc, x, c1, c2, c3, c4, k31, k41, k51)
    NIR_values.append(NIR)
for index, x in enumerate(power[conc][1]):
    _, blue = compute_values_for_x(conc, x, c1, c2, c3, c4, k31, k41, k51)
    blue_total_values.append(blue)



In [53]:
import json
fig = go.Figure()

# Add the main trace (mean curve)
fig.add_trace(go.Scatter(x=power[conc][0], y=NIR_values, mode='lines+markers', name='Mean_NIR_Simulation'))
fig.add_trace(go.Scatter(x=power[conc][1], y=blue_total_values, mode='lines+markers', name='Mean_blue_Simulation'))

# Add the uncertainty shade trace
# fig.add_trace(go.Scatter(
#     x=power[conc][0] + power[conc][0][::-1],
#     y=[i+j for i, j in zip(nir_mean_values,nir_std_values)] + [i-j for i, j in zip(nir_mean_values,nir_std_values)][::-1],
#     fill='toself',
#     fillcolor='rgba(170, 68, 153, 0.2)',
#     line=dict(color='rgba(255,255,255,0)'),
#     name='NIR_simulation'
# ))

# fig.add_trace(go.Scatter(
#     x=power[conc][1] + power[conc][1][::-1],
#     y=[i+j for i, j in zip(blue_mean_values,blue_std_values)] + [i-j for i, j in zip(blue_mean_values,blue_std_values)][::-1],
#     fill='toself',
#     fillcolor='rgba(170, 68, 153, 0.2)',
#     line=dict(color='rgba(255,255,255,0)'),
#     name='NIR_simulation'
# ))


fig.update_xaxes(title_font=dict(size=18, color='black'),tickfont=dict(size=14),showgrid=True, gridwidth=2, gridcolor='white', dtick=2000)
fig.update_yaxes(title_font=dict(size=18, color='black'),tickfont=dict(size=14),showgrid=True, gridwidth=2, gridcolor='white')

# ... Your existing layout and formatting ...

fig.update_layout(
    title_text='NaYbF4: (1-x)% Yb, x% Tm',  # Main Title
    title_font=dict(size=24, family="Arial, sans-serif", color='royalblue'),
    width=800, height=600,
    xaxis_title='Power Density (W/cm^2)',
    yaxis_title='Intensity(kpps)',
    title_x=0.42,  # Center the main title
    title_y=0.95,  # Adjust the vertical position of the main title
    showlegend=True, margin=dict(l=20, r=20, b=20, t=70),
    legend=dict(
        font=dict(
            size=20,
            family="Arial, sans-serif"
        )
    )
)

with open(f'./../synthetic_data/syn_{conc}.json', 'r') as f:
    loaded_json_data = f.read()
loaded_data = json.loads(loaded_json_data)
exp_blue_total = loaded_data.get('blue', [])
exp_NIR = loaded_data.get('NIR', [])
fig.add_trace(go.Scatter(x= power[conc][0], y= exp_NIR, mode='markers', name='NIR_exp', marker=dict(color='rgb(170, 68, 153)', size=10)))
fig.add_trace(go.Scatter(x= power[conc][1], y= exp_blue_total, mode='markers', name='Blue_exp', marker=dict(color='#0099C6', size=10)))

# Show the plot
fig.show()
# print('hihi')

In [54]:
fig = go.Figure()

# Adding subplots for each parameter
# c1_exps, c2_exps, c3_exps, c4_exps, k31_exps, k41_exps, k51_exps = c1, c2, c3, c4, k31, k41, k51
params = [c1_exps, c2_exps, c3_exps, c4_exps, k31_exps, k41_exps, k51_exps]
param_names = ["c1", "c2", "c3", "c4", "k31", "k41", "k51"]
initials = [2, -1, 0, 2, 0, 2, 0]

for param, name in zip(params, param_names):
    gd_value = params[param_names.index(name)]  # Assuming the order matches
    fig.add_trace(go.Scatter(
        x=[name],
        y=[gd_value],
        mode='markers',
        marker=dict(size=10, color='red'),
        name=f'{name} GD results'
    ))

    truth = loaded_data.get('param', [])
    truth_value = truth[param_names.index(name)]
    fig.add_trace(go.Scatter(
        x=[name],
        y=[truth_value],
        mode='markers',
        marker=dict(size=10, color='blue'),
        name=f'{name} input'
    ))

    init_value = initials[param_names.index(name)]
    fig.add_trace(go.Scatter(
        x=[name],
        y=[init_value],
        mode='markers',
        marker=dict(size=5, color='green'),
        name=f'{name} initial'
    ))

fig.update_xaxes(showgrid=True, gridwidth=2, gridcolor='white')
fig.update_yaxes(showgrid=True, gridwidth=2, gridcolor='white')

fig.update_layout(
        title=f"Parameter exponents",
        title_font=dict(size=24, family="Courier New, monospace"),
        title_x=0.5,  # center
        title_y=0.9,
        width=640, height=560,
        yaxis_title="Parameter Exponent Value",
        xaxis_title="Seven Parameter Exponents",
        xaxis=dict(
            title_font=dict(size=22, family="Courier New, monospace"),
            tickfont=dict(size=22, family="Courier New, monospace")
        ),
        yaxis=dict(
            title_font=dict(size=22, family="Courier New, monospace"),
            tickfont=dict(size=22, family="Courier New, monospace")
        )
    )


fig.show()